# Procesamiento de Texto

In [9]:
%pip install unidecode
%pip install spacy
!python -m spacy download es_core_news_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------- ------------------ 6.8/12.9 MB 35.8 MB/s eta 0:00:01
     --------------------------------------  12.6/12.9 MB 30.9 MB/s eta 0:00:01
     ---------------------------------------- 12.9/12.9 MB 28.2 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [1]:
%pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 23.9 MB/s  0:00:00

   ---------------------------------------- 0/3 [regex]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
   ------------- -------------------------- 1/3 [joblib]
  

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Pasos

### 1. Limpieza

El objetivo de este paso es hacer un reemplazo de simbologia del español, puntuación, etc.
Se podría hacer uso de .replace() de python. Sin embargo se trabaja con expresiones regulares, en este caso la libreria "re".

In [2]:
import re

def limpiar(texto):
    puntuación = r'[,;.:¡!¿?@#$%&[\](){}<>~=+\-*/|\\_^`"\']'
    
    # signos de puntuación
    texto = re.sub(puntuación, ' ', texto)
    
    # dígitos [0-9]
    # Se usa \d para representar dígitos usualmente.
    texto = re.sub('[0-9]', ' ', texto)

    return texto

ejemplo = "¡Hola, mundo! ¿Cómo estás? 1234"
resultado = limpiar(ejemplo)
resultado


' Hola  mundo   Cómo estás      '

### 2. Paso

Ejemplo:
- 13/03/30 -> trece de marzo de dos mil treinta
- DC -> departamento de computación

Es necesario ahora normalizar, osea, para el español eliminar tildes, convertir todo a minusculas.

Si bien podemos hacerlo de manera manual, se puede usar "unicode" para lograrlo en caso que se tenga caracteres chinos o letras extranjeras.

In [7]:
def normalizar(texto):

    # todo a minúsculas
    texto = texto.lower()

    # tildes y diacríticas
    texto = re.sub('á', 'a', texto)
    texto = re.sub('é', 'e', texto)
    texto = re.sub('í', 'i', texto)
    texto = re.sub('ó', 'o', texto)
    texto = re.sub('ú', 'u', texto)
    texto = re.sub('ü', 'u', texto)
    texto = re.sub('ñ', 'n', texto)

    return texto

resultado_normalizado = normalizar(resultado)
resultado_normalizado

' hola  mundo   como estas      '

In [3]:
# Opcion 2 Unicode

from unidecode import unidecode

resultado_normalizado2 = unidecode(resultado)
resultado_normalizado2

' Hola  mundo   Como estas      '

### Paso 3: Finalmente, tokenización
Separar el texto en partes mas pequeñas. Hay muchas modalidades y hay que tener mucho cuidado, sobretodo en situación donde la puntuación puede ser seperador. En ingles tenemos "it's" , que podría separarse como it - 's.

In [5]:
def tokenizar(texto):
    # IMPORTANTE: podría devolver una lista vacía
    return [tóken for tóken in texto.split()]
    
ejemplo_tokenizado = tokenizar(resultado_normalizado2)
ejemplo_tokenizado

['Hola', 'mundo', 'Como', 'estas']

Entonces, ahora usando spacy

In [10]:
import spacy

nlp = spacy.load('es_core_news_sm')

doc = nlp('Esto es una frase.')

print([tóken.text for tóken in doc])

['Esto', 'es', 'una', 'frase', '.']


### Stemming

Stem, de raíz, reduce la inflección de las palabras, mapeando un grupo de palabras a la misma raíz, sin importar si la raíz es una palabras válida en el lenguaje.

### Lemmatization

A diferencia del stemming, la lematización reduce las palabras inflexadas a palabras que pertenecen al lenguaje. La raíz pasa a llamarse lema.

# Trabajando con NLTK

In [3]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords.words('spanish')[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acca4\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']

#  Ejemplo en Python con spaCy

In [1]:
import spacy

# 1. Cargar modelo en español
nlp = spacy.load("es_core_news_sm")

# 2. Texto de ejemplo
texto = "¡Hola! Estoy aprendiendo procesamiento de lenguaje natural con Python."

# 3. Procesar texto
doc = nlp(texto)

# 4. Tokenización
tokens = [token.text for token in doc]
print("Tokens:", tokens)

# 5. Lemmatización
lemmas = [token.lemma_ for token in doc]
print("Lemmas:", lemmas)

# 6. Filtrar solo palabras significativas (sin puntuación ni stopwords)
tokens_filtrados = [token.text for token in doc if not token.is_stop and token.is_alpha]
print("Tokens filtrados:", tokens_filtrados)


Tokens: ['¡', 'Hola', '!', 'Estoy', 'aprendiendo', 'procesamiento', 'de', 'lenguaje', 'natural', 'con', 'Python', '.']
Lemmas: ['¡', 'Hola', '!', 'estar', 'aprender', 'procesamiento', 'de', 'lenguaje', 'natural', 'con', 'Python', '.']
Tokens filtrados: ['Hola', 'aprendiendo', 'procesamiento', 'lenguaje', 'natural', 'Python']


# Ejemplo rápido con HuggingFace Tokenizers (sub-palabras)

In [2]:
%pip install transformers

   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------------------- -------------------- 5.5/11.3 MB 28.8 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.3 MB 28.5 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 26.5 MB/s  0:00:00
   ---------------------------------------- 0.0/561.5 kB ? eta -:--:--
   ---------------------------------------- 561.5/561.5 kB 20.8 MB/s  0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 25.3 MB/s  0:00:00

   ----- ---------------------------------- 1/7 [pyyaml]
   ----- ---------------------------------- 1/7 [pyyaml]
   ----------- ---------------------------- 2/7 [fsspec]
   ----------- ---------------------------- 2/7 [fsspec]
   ----------- ---------------------------- 2/7 [fsspec]
   ----------- ---------------------------- 2/7 [fsspec]
   ----------- ---------------------------- 2/7 [fsspec]
   --

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
from transformers import AutoTokenizer

# Tokenizador preentrenado (multilingüe)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

texto = "¡Hola! Estoy aprendiendo procesamiento de lenguaje natural con Python."

# Tokenización por sub-palabras
tokens = tokenizer.tokenize(texto)
ids = tokenizer.convert_tokens_to_ids(tokens)

print("Tokens:", tokens)
print("IDs:", ids)


c:\Users\acca4\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\acca4\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\acca4\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https

Tokens: ['¡', 'Ho', '##la', '!', 'Esto', '##y', 'apre', '##ndi', '##endo', 'procesa', '##miento', 'de', 'lenguaje', 'natural', 'con', 'Python', '.']
IDs: [199, 20220, 10330, 106, 24165, 10157, 87438, 17938, 17560, 103862, 16085, 10104, 65621, 13409, 10173, 47294, 119]
